In [2]:
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# from utils_subdivision.gen_distribution_single_plots import analyze_phases
# from utils_subdivision.gen_distribution_subplot import analyze_single_type    # plot_combined_results
# from utils_subdivision.gen_distribution_merged_plot import plot_merged
from utils_dot_plot.drum_single import analyze_phases
# from utils_dot_plot.drum_merged import plot_merged_per_mode

from utils_subdivision.gen_distribution_subplot import analyze_single_type
from utils_dot_plot.kinematic_dot_plot import *
from utils_dot_plot.drum_merged import *


base_output_dir = "output_dot_plots"

# Generate Dance dot plot by Group, Individual or Audience

In [ ]:
m_idx = 0
mode = ["group", "individual", "audience"]
dance_mode = mode[m_idx]

with open('data/selected_piece_list.pkl', 'rb') as f:
    piece_list = pickle.load(f)
    
for file_name in piece_list:
    print(file_name)
    cycles_csv_path = f"data/virtual_cycles/{file_name}_C.csv"
    
    dmode_path = f"data/dance_modes_ts/{file_name}_{dance_mode}.pkl"
    if not os.path.exists(dmode_path):
        continue
    
    left_onset_path = f"data/logs_v4_0.007_foot_jun3/{file_name}_T/onset_info/{file_name}_T_left_foot_onsets.csv"
    right_onset_path = f"data/logs_v4_0.007_foot_jun3/{file_name}_T/onset_info/{file_name}_T_right_foot_onsets.csv"
    
    left_onsets = pd.read_csv(left_onset_path)["time_sec"].values
    right_onsets = pd.read_csv(right_onset_path)["time_sec"].values
    
    if os.path.exists(dmode_path):
        with open(dmode_path, "rb") as f:
            dance_mode_time_segments = pickle.load(f)
            
        fig, ax, _ = plot_foot_onsets_stacked(
            file_name=file_name,
            dance_mode=dance_mode,
            cycles_csv_path=cycles_csv_path,
            left_onsets=left_onsets,
            right_onsets=right_onsets,
            dance_mode_time_segments=dance_mode_time_segments,
            figsize=(10, 3),
            dpi=200,
            use_window=True,
            legend_flag=False
        )
        
        save_dir = os.path.join(base_output_dir, "dance", dance_mode)
        os.makedirs(save_dir, exist_ok=True)
        
        save_path = os.path.join(save_dir, f"{file_name}_{dance_mode}_merged.png")
        plt.savefig(save_path, bbox_inches='tight')  # Add bbox_inches='tight' to prevent label clipping
        plt.close()
    # break

### Merged Drum Dot Plot - All modes combined

In [30]:
def plot_merged_stacked_all_modes(
    file_name,
    cycles_csv_path,
    onsets_csv_path,
    dance_mode_time_segments_all,  # Dictionary containing time segments for all modes
    figsize=(10, 3),
    dpi=200,
    use_window=True,
    legend_flag=True
):
    """Create a single plot showing combined drum onset analysis for all modes for a single file."""
    
    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    vertical_ranges = {
        'Dun': (1, 6),
        'J1': (8, 13),
        'J2': (15, 20)
    }

    # Fixed colors for each drum type
    drum_colors = {
        'Dun': '#1f77b4',   # blue
        'J1': '#d62728',    # red
        'J2': '#2ca02c'     # green
    }

    # Initialize combined data structure
    onset_types = ['Dun', 'J1', 'J2']
    drum_phases_kde = {onset_type: {"phases": [], "y_scaled": []} for onset_type in onset_types}
    
    # Process each mode
    for mode, dance_mode_time_segments in dance_mode_time_segments_all.items():
        # Get cycles data
        cycles_df = pd.read_csv(cycles_csv_path)
        cycle_times = cycles_df['Virtual Onset'].values
        
        # Get onsets data
        onsets_df = pd.read_csv(onsets_csv_path)
        
        # Process each onset type
        for onset_type in onset_types:
            # Get onset times for this type
            onset_times = onsets_df[onset_type].dropna().values  # Remove NaN values
            
            # Process each time segment
            for segment in dance_mode_time_segments:
                start_time, end_time = segment  # Unpack the tuple
                
                # Get cycles in this segment
                segment_cycles = cycle_times[
                    (cycle_times >= start_time) & 
                    (cycle_times <= end_time)
                ]
                
                if len(segment_cycles) == 0:
                    continue
                
                # Get onsets in this segment
                segment_onsets = onset_times[
                    (onset_times >= start_time) & 
                    (onset_times <= end_time)
                ]
                
                if len(segment_onsets) == 0:
                    continue
                
                # Calculate phases for each onset
                for onset_time in segment_onsets:
                    # Find the cycle containing this onset
                    cycle_mask = (cycle_times <= onset_time)
                    if not any(cycle_mask):
                        continue
                    
                    cycle_start = cycle_times[cycle_mask][-1]  # Last cycle before onset
                    
                    # Find the next cycle
                    next_cycle_mask = (cycle_times > onset_time)
                    if any(next_cycle_mask):
                        cycle_end = cycle_times[next_cycle_mask][0]
                    else:
                        cycle_end = cycle_times[-1]
                    
                    # Calculate phase
                    phase = (onset_time - cycle_start) / (cycle_end - cycle_start)
                    
                    # Handle cycle wrapping
                    if phase > 0.98:  # If phase is closer to next cycle
                        phase = phase - 1.0
                    
                    # Add to combined data
                    drum_phases_kde[onset_type]["phases"].append(phase)
                    drum_phases_kde[onset_type]["y_scaled"].append(len(drum_phases_kde[onset_type]["phases"]))

    # Plot the combined data
    for onset_type, color in drum_colors.items():
        phases = np.array(drum_phases_kde[onset_type]["phases"])
        y_scaled = np.array(drum_phases_kde[onset_type]["y_scaled"])
        
        if len(phases) == 0:
            continue
            
        # Normalize y_scaled values to fit within the vertical range
        y_min, y_max = vertical_ranges[onset_type]
        y_scaled = y_min + (y_scaled - np.min(y_scaled)) * (y_max - y_min) / (np.max(y_scaled) - np.min(y_scaled))
        
        # Plot scatter
        ax.scatter(phases * 400,
                  y_scaled,
                  s=5, alpha=0.4,
                  color=color,
                  label=f'{onset_type}')

    # Calculate and plot combined KDE
    all_phases = []
    for onset_type in onset_types:
        all_phases.extend(drum_phases_kde[onset_type]["phases"])
    
    if len(all_phases) > 0:
        kde_xx, kde_h = kde_estimate(np.array(all_phases), SIG=0.01)
        
        # Only plot the region that maps to the x-axis
        mask = (kde_xx * 400 >= -33) & (kde_xx * 400 <= 400)
        kde_xx_plot = kde_xx[mask]
        kde_h_plot = kde_h[mask]
        
        if np.max(kde_h_plot) > 0:
            kde_scaled = -5 + (5 * kde_h_plot / np.max(kde_h_plot))
            ax.fill_between(kde_xx_plot * 400, -5, kde_scaled, alpha=0.3, color='purple', label='Combined KDE')

    # Subdivision lines
    for subdiv in range(1, 13):
        color = get_subdiv_color(subdiv)
        x_pos = ((subdiv-1) * 400) / 12
        
        if subdiv in [1, 4, 7, 10]:
            ax.vlines(x_pos, -5.5, 20.5, color=color, linestyle='-', linewidth=1.5, alpha=0.7)
        else:
            ax.vlines(x_pos, -5.5, 20.5, color=color, linestyle='--', linewidth=1, alpha=0.3)

    # Styling
    xtick = [0, 100, 200, 300, 400]
    xtick_labels = [1, 2, 3, 4, 5]
    
    ax.set_xticks(xtick)
    ax.set_xticklabels(xtick_labels)
    ax.set_xlim(-33, 400)
    ax.set_xlabel('Beat span')
    
    ax.set_ylim(-5.5, 20.5)
    ax.set_yticks([3, 10, 17])
    ax.set_yticklabels(['Dun', 'J1', 'J2'])
    ax.set_ylabel('Drum')
    ax.grid(True, alpha=0.3)

    # Title & legend
    title = f'File: {file_name} | All Modes Combined'
    # title += f' | Combined from {len(dance_mode_time_segments_all)} modes'
    ax.set_title(title, pad=10)
    
    if legend_flag:
        ax.legend(loc='upper left', framealpha=0.4, fontsize=6)

    return fig, ax, drum_phases_kde

In [ ]:
# Load the piece list
with open('data/selected_piece_list.pkl', 'rb') as f:
    piece_list = pickle.load(f)

# Loop over each file in the piece list
for file_name in piece_list:
    print(f"Processing {file_name}...")
    
    # Define paths
    cycles_csv_path = f"data/virtual_cycles/{file_name}_C.csv"
    onsets_csv_path = f"data/drum_onsets/{file_name}.csv"
    
    # Check if required files exist
    if not (os.path.exists(cycles_csv_path) and os.path.exists(onsets_csv_path)):
        print(f"Skipping {file_name} - missing required files")
        continue
    
    # Load all modes' time segments
    dance_mode_time_segments_all = {}
    for mode in ["group", "individual", "audience"]:
        dmode_path = f"data/dance_modes_ts/{file_name}_{mode}.pkl"
        if os.path.exists(dmode_path):
            with open(dmode_path, "rb") as f:
                dance_mode_time_segments_all[mode] = pickle.load(f)
    
    # Skip if no mode data available
    if not dance_mode_time_segments_all:
        print(f"Skipping {file_name} - no mode data available")
        continue
    
    # try:
    # Call the modified function
    fig, ax, drum_phases_kde = plot_merged_stacked_all_modes(
        file_name=file_name,
        cycles_csv_path=cycles_csv_path,
        onsets_csv_path=onsets_csv_path,
        dance_mode_time_segments_all=dance_mode_time_segments_all,
        figsize=(10, 3),
        dpi=200,
        use_window=True,
        legend_flag=False
    )
    
    # Save the figure
    save_dir = os.path.join(base_output_dir, "drum_merged", "all_modes")
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f"{file_name}_all_modes_merged.png")
    plt.savefig(save_path, bbox_inches='tight', dpi=200)
    plt.close()
    
    print(f"Successfully processed {file_name}")
        
    # except Exception as e:
    #     print(f"Error processing {file_name}: {str(e)}")
    #     continue

print("Processing complete!")

## Dance dot plot - All modes combined

In [42]:
def plot_foot_onsets_stacked_all_modes(
    file_name,
    cycles_csv_path,
    left_onset_path,
    right_onset_path,
    dance_mode_time_segments_all,  # Dictionary containing time segments for all modes
    figsize=(10, 3),
    dpi=200,
    use_window=True,
    legend_flag=True
):
    """Create a single plot showing combined foot onset analysis for all modes for a single file."""
    
    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    vertical_ranges = {
        'left': (1, 6),
        'right': (8, 13)
    }

    # Fixed colors for feet
    foot_colors = {
        'left': '#1f77b4',   # blue
        'right': '#d62728'   # red
    }

    # Initialize combined data structure
    foot_types = ['left', 'right']
    foot_phases_kde = {foot_type: {"phases": [], "y_scaled": []} for foot_type in foot_types}
    
    # Load foot onset data
    left_onsets = pd.read_csv(left_onset_path)["time_sec"].values
    right_onsets = pd.read_csv(right_onset_path)["time_sec"].values
    
    # Process each mode
    for mode, dance_mode_time_segments in dance_mode_time_segments_all.items():
        # Get cycles data
        cycles_df = pd.read_csv(cycles_csv_path)
        cycle_times = cycles_df['Virtual Onset'].values
        
        # Process each foot type
        for foot_type, color in foot_colors.items():
            # Get onset times for this foot
            onset_times = left_onsets if foot_type == 'left' else right_onsets
            
            # Process each time segment
            for segment in dance_mode_time_segments:
                start_time, end_time = segment  # Unpack the tuple
                
                # Get cycles in this segment
                segment_cycles = cycle_times[
                    (cycle_times >= start_time) & 
                    (cycle_times <= end_time)
                ]
                
                if len(segment_cycles) == 0:
                    continue
                
                # Get onsets in this segment
                segment_onsets = onset_times[
                    (onset_times >= start_time) & 
                    (onset_times <= end_time)
                ]
                
                if len(segment_onsets) == 0:
                    continue
                
                # Calculate phases for each onset
                for onset_time in segment_onsets:
                    # Find the cycle containing this onset
                    cycle_mask = (cycle_times <= onset_time)
                    if not any(cycle_mask):
                        continue
                    
                    cycle_start = cycle_times[cycle_mask][-1]  # Last cycle before onset
                    
                    # Find the next cycle
                    next_cycle_mask = (cycle_times > onset_time)
                    if any(next_cycle_mask):
                        cycle_end = cycle_times[next_cycle_mask][0]
                    else:
                        cycle_end = cycle_times[-1]
                    
                    # Skip if cycle_start equals cycle_end
                    if cycle_end == cycle_start:
                        continue
                    
                    # Calculate phase
                    phase = (onset_time - cycle_start) / (cycle_end - cycle_start)
                    
                    # Handle cycle wrapping
                    # if phase > 0.5:  # If phase is closer to next cycle
                    #     phase = phase - 1.0
                    
                    # Add to combined data
                    foot_phases_kde[foot_type]["phases"].append(phase)
                    foot_phases_kde[foot_type]["y_scaled"].append(len(foot_phases_kde[foot_type]["phases"]))

    # Plot the combined data
    for foot_type, color in foot_colors.items():
        phases = np.array(foot_phases_kde[foot_type]["phases"])
        y_scaled = np.array(foot_phases_kde[foot_type]["y_scaled"])
        
        if len(phases) == 0:
            continue
            
        # Normalize y_scaled values to fit within the vertical range
        y_min, y_max = vertical_ranges[foot_type]
        y_scaled = y_min + (y_scaled - np.min(y_scaled)) * (y_max - y_min) / (np.max(y_scaled) - np.min(y_scaled))
    
        # Plot scatter
        ax.scatter(phases * 400,
                  y_scaled,
                  s=5, alpha=0.4,
                  color=color,
                  label=f'{foot_type.capitalize()} Foot')

    # Calculate and plot combined KDE
    all_phases = []
    for foot_type in foot_types:
        all_phases.extend(foot_phases_kde[foot_type]["phases"])
    
    if len(all_phases) > 0:
        kde_xx, kde_h = kde_estimate(np.array(all_phases), SIG=0.01)
        
        # Only plot the region that maps to the x-axis
        mask = (kde_xx * 400 >= -33) & (kde_xx * 400 <= 400)
        kde_xx_plot = kde_xx[mask]
        kde_h_plot = kde_h[mask]
        
        if np.max(kde_h_plot) > 0:
            kde_scaled = -5 + (5 * kde_h_plot / np.max(kde_h_plot))
            ax.fill_between(kde_xx_plot * 400, -5, kde_scaled, alpha=0.3, color='purple', label='Combined KDE')

    # Subdivision lines
    for subdiv in range(1, 13):
        color = get_subdiv_color(subdiv)
        x_pos = ((subdiv-1) * 400) / 12
        
        if subdiv in [1, 4, 7, 10]:
            ax.vlines(x_pos, -5.5, 13.5, color=color, linestyle='-', linewidth=1.5, alpha=0.7)
        else:
            ax.vlines(x_pos, -5.5, 13.5, color=color, linestyle='--', linewidth=1, alpha=0.3)

    # Styling
    xtick = [0, 100, 200, 300, 400]
    xtick_labels = [1, 2, 3, 4, 5]
    
    ax.set_xticks(xtick)
    ax.set_xticklabels(xtick_labels)
    ax.set_xlim(-33, 400)
    ax.set_xlabel('Beat span')
    
    ax.set_ylim(-5.5, 13.5)
    ax.set_yticks([3, 10])
    ax.set_yticklabels(['LF', 'RF'])
    ax.set_ylabel('Foot')
    ax.grid(True, alpha=0.3)

    # Title & legend
    title = f'File: {file_name} | All Modes Combined'
    # title += f' | Combined from {len(dance_mode_time_segments_all)} modes'
    ax.set_title(title, pad=10)
    
    if legend_flag:
        ax.legend(loc='upper left', framealpha=0.4, fontsize=6)

    return fig, ax, foot_phases_kde

In [43]:
# Load the piece list
with open('data/selected_piece_list.pkl', 'rb') as f:
    piece_list = pickle.load(f)

# Loop over each file in the piece list
for file_name in piece_list:
    print(f"Processing {file_name}...")
    
    # Define paths
    cycles_csv_path = f"data/virtual_cycles/{file_name}_C.csv"
    left_onset_path = f"data/logs_v4_0.007_foot_jun3/{file_name}_T/onset_info/{file_name}_T_left_foot_onsets.csv"
    right_onset_path = f"data/logs_v4_0.007_foot_jun3/{file_name}_T/onset_info/{file_name}_T_right_foot_onsets.csv"
    
    # Check if required files exist
    if not (os.path.exists(cycles_csv_path) and 
            os.path.exists(left_onset_path) and 
            os.path.exists(right_onset_path)):
        print(f"Skipping {file_name} - missing required files")
        continue
    
    # Load all modes' time segments
    dance_mode_time_segments_all = {}
    for mode in ["group", "individual", "audience"]:
        dmode_path = f"data/dance_modes_ts/{file_name}_{mode}.pkl"
        if os.path.exists(dmode_path):
            with open(dmode_path, "rb") as f:
                dance_mode_time_segments_all[mode] = pickle.load(f)
    
    # Skip if no mode data available
    if not dance_mode_time_segments_all:
        print(f"Skipping {file_name} - no mode data available")
        continue
    

    # Call the modified function
    fig, ax, foot_phases_kde = plot_foot_onsets_stacked_all_modes(
        file_name=file_name,
        cycles_csv_path=cycles_csv_path,
        left_onset_path=left_onset_path,
        right_onset_path=right_onset_path,
        dance_mode_time_segments_all=dance_mode_time_segments_all,
        figsize=(10, 3),
        dpi=200,
        use_window=True,
        legend_flag=False
    )
    
    # Save the figure
    save_dir = os.path.join(base_output_dir, "dance", "all_modes")
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f"{file_name}_all_modes_merged.png")
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    
    print(f"Successfully processed {file_name}")
        


print("Processing complete!")

Processing BKO_E1_D1_01_Suku...
Successfully processed BKO_E1_D1_01_Suku
Processing BKO_E1_D1_02_Maraka...
Successfully processed BKO_E1_D1_02_Maraka
Processing BKO_E1_D1_03_Wasulunka...
Successfully processed BKO_E1_D1_03_Wasulunka
Processing BKO_E1_D1_06_Manjanin...
Successfully processed BKO_E1_D1_06_Manjanin
Processing BKO_E1_D1_07_Suku...
Successfully processed BKO_E1_D1_07_Suku
Processing BKO_E1_D1_08_Suku...
Successfully processed BKO_E1_D1_08_Suku
Processing BKO_E1_D2_03_Suku...
Successfully processed BKO_E1_D2_03_Suku
Processing BKO_E1_D2_04_Maraka...
Successfully processed BKO_E1_D2_04_Maraka
Processing BKO_E1_D2_05_Wasulunka...
Successfully processed BKO_E1_D2_05_Wasulunka
Processing BKO_E1_D5_01_Maraka...
Successfully processed BKO_E1_D5_01_Maraka
Processing BKO_E1_D5_04_Suku...
Successfully processed BKO_E1_D5_04_Suku
Processing BKO_E2_D3_01_Maraka...
Successfully processed BKO_E2_D3_01_Maraka
Processing BKO_E2_D3_02_Suku...
Successfully processed BKO_E2_D3_02_Suku
Process